# Bioimage Analysis Workflow 1 - Leaf Cellular Morphological Analysis (1)

## Loading and Handling Image Data

In [ ]:
# (1) Specify the file path

# Create a string variable "filepath" with the path to the file you'd like to load
filepath = r'example_data\Leaf_PM_Nuclei.tif'

In [ ]:
# (2) Load the image

# Import the function 'imread' from the module 'skimage.io'
from skimage.io import imread

# Load the image and store it in a variable
img = imread(filepath)

In [ ]:
# (3) Check variable type, file shape and data type

# Print that 'img' is a variable of type 'ndarray' - use Python's built-in function 'type'.
print("Loaded array is of type:", type(img))

# Print the shape of the array by looking at its 'shape' attribute. 
print("Loaded array has shape:", img.shape)

# Print the datatype of the individual numbers in the array. You can use the array attribute 'dtype' to do so.
print("Loaded values are of type:", img.dtype)

In [ ]:
# (4) Split channel

# From the image shape, it was shown that the image has two channels - Split channels
img_ch0 = img[0,:,:]
img_ch1 = img[1,:,:]

In [ ]:
# (5) Show images

# First import plotting module matplotlib.pyplot as plt
import matplotlib.pyplot as plt

# Show images with the plt.figure/imshow function
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img_ch0, interpolation='none', cmap = 'gray')
plt.title('Channel 0')
plt.subplot(122)
plt.imshow(img_ch1, interpolation='none', cmap = 'gray')
plt.title('Channel 1')
plt.show()

In [ ]:
# (6) Crop image for processing

# Define the x & y range for cropping
x_start = 500
x_end = 1500
y_start = 500
y_end = 1500

# Crop
img_ch0_crop = img_ch0[y_start:y_end,x_start:x_end]
img_ch1_crop = img_ch1[y_start:y_end,x_start:x_end]

In [ ]:
# (7) Display cropped images
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img_ch0_crop, interpolation='none', cmap = 'gray')
plt.title('Channel 0')
plt.subplot(122)
plt.imshow(img_ch1_crop, interpolation='none', cmap = 'gray')
plt.title('Channel 1')
plt.show()

In [ ]:
# (8) BONUS: Ch0 image has a high dynamic range. Scale the intensity of Ch0 image by log operation

# importing the package NumPy, which enables the manipulation of numerical arrays
import numpy as np

# Replace zeros by ones
img_ch0_crop_nozero = np.maximum(img_ch0_crop, 1)

# Apply log operation
img_ch0_crop_nozero_log = np.log(img_ch0_crop_nozero)

# Show log adjusted image
plt.imshow(img_ch0_crop_nozero_log, interpolation='none', cmap = 'gray')

### Compare JPEG and TIF

In [ ]:
filepath_jpg = r'example_data/Leaf_PM_Nuclei.jpg'
img_jpg = imread(filepath_jpg)

In [ ]:
img_jpg.shape

In [ ]:
x_start = 480
x_end = 520
y_start = 480
y_end = 520
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img_ch1[y_start:y_end, x_start:x_end], interpolation='none')
plt.title('Image saved by TIF')
plt.subplot(122)
plt.imshow(img_jpg[y_start:y_end, x_start:x_end], interpolation='none')
plt.title('Image saved by JPEG')
plt.show()

## Preprocessing

Apply gaussian filter to the image

In [ ]:
# Gaussian Filtering
# import n-D image analysis module scipy.ndimage as ndi
import scipy.ndimage as ndi

# set sigma value
sigma = 1.5

# Appy gaussian filter using ndi.gaussian_filter
img_smooth = ndi.gaussian_filter(img_ch1_crop, sigma)

# Show image
plt.imshow(img_smooth, interpolation='none', cmap='gray')

## Global Thresholding

In [ ]:
# (i) Create a variable for a manually set threshold, which should be an integer

# This can be changed later to find a suitable value.
thresh = 70

In [ ]:
# (ii) Perform thresholding on the smoothed image

# Use relational (Boolean) expressions for thresholding
wall = img_smooth > thresh

# Check the dtype of your thresholded image
print(wall.dtype)

In [ ]:
# (iii) Visualize the result

plt.imshow(wall, interpolation='none', cmap='gray')

In [ ]:
# (iv) Try out different thresholds to find the best one

# Prepare widget
from ipywidgets import interact
@interact(thresh=(10,250,10))
def select_threshold(thresh=100):
    
    # Thresholding
    ### ADAPT THIS: Change 'img_smooth' into the variable you stored the smoothed image in!
    wall = img_smooth > thresh
    
    # Visualization
    plt.figure(figsize=(7,7))
    plt.imshow(wall, interpolation='none', cmap='gray')
    plt.show()

In [ ]:
# (v) Perfom automated threshold detection with Otsu's method

# Import
from skimage.filters.thresholding import threshold_otsu

# Calculate and apply threshold
thresh = threshold_otsu(img_smooth)
wall = img_smooth > thresh
    
# Visualization
plt.imshow(wall, interpolation='none', cmap='gray')

In [ ]:
# (vi) BONUS: Did you notice the 'try_all_threshold' function?

from skimage.filters.thresholding import try_all_threshold
fig = try_all_threshold(img_smooth, figsize=(10,10), verbose=False)

## Local (Adaptive) Thresholding

Create a background image by mean filtering, then perform local thresholding

Hint for mean filtering: https://scikit-image.org/docs/stable/auto_examples/filters/plot_rank_mean.html

In [ ]:
# (1) Create a disk-shaped foot-print (Kernel) and asign it to a new variable.

# Import module disk from skimage.morphology
from skimage.morphology import disk

# Create footprint for mean filtering
footprint = disk(5)

# Visulize footprint
plt.imshow(footprint)

In [ ]:
# (2) Create background by mean filtering, then visulize

# Import rank from skimage.filters
from skimage.filters import rank 

# Apply mean filter to create background
background = rank.mean(img_smooth, footprint= footprint)

# Show background
plt.imshow(background, interpolation='none', cmap='gray')
plt.title('Background')

In [ ]:
# (3) Perform local thresholding
# Threshold the Gaussian-smoothed original image (img_smooth) against the background image created in step 2 
#      using a relational expression
cell_wall = img_smooth > background
plt.figure(figsize=(10,10))
plt.imshow(cell_wall, interpolation='none', cmap='gray')
plt.title('Cell Wall')

In [ ]:
## LEGACY: local thresholding == background subtraction -> global thresholding
# # (3) Perform background subtraction
# img_bgd_sub = img_smooth.astype(float) - background.astype(float)
# plt.figure(figsize=(5,5))
# plt.imshow(img_bgd_sub, interpolation='none', cmap='gray')
# plt.title('Background Subtracted')

# # (4) Thresholding
# from skimage.filters.thresholding import threshold_mean
# thresh = threshold_mean(img_bgd_sub)
# wall = img_bgd_sub > thresh
# plt.figure(figsize=(5,5))
# plt.imshow(wall, interpolation='none', cmap = 'gray')

## Improving Masks with Binary Morphology

In [ ]:
# (1) Get rid of speckles using binary hole filling

wall_holefilled = ~ndi.binary_fill_holes(~cell_wall)
plt.figure(figsize=(5,5))
plt.imshow(wall_holefilled, interpolation='none', cmap='gray')

In [ ]:
# (2) Closing the gaps in the membrane by dilation

# Create a SE for the binary operation with disk()
r = 2
SE = disk(r)

# Perform dilation with the python function ndi.binary_dilation
wall_dilated = ndi.binary_dilation(wall_holefilled, structure=SE)

# Now visulize the result
plt.imshow(wall_dilated, cmap = 'gray');

In [ ]:
# (3) Restore the membrane shape by erosion

# Using the same SE as before, perform erosion with ndi.binary_erosion
wall_eroded = ndi.binary_erosion(wall_dilated, structure=SE)

# Now visulize the result
plt.imshow(wall_eroded, cmap = 'gray');

In [ ]:
# (4) [BONUS 1] If you pay close attention, you will notice that some of these operations introduce 
# artefacts at the image boundaries. Can you come up with a way of solving this? (Hint: 'np.pad')
# [BONUS 2] You just did dilation and erosion with the same SE. These two operations
# combined together is called "closing". Try ndi.binary_closing to do the same thing in one line
import numpy as np
r = 2
SE = disk(r)
pad_size = r + 1
wall_padded = np.pad(wall_holefilled, pad_size, mode='reflect')
wall_final = ndi.binary_closing(wall_padded, structure=SE)
wall_final = wall_final[pad_size:-pad_size, pad_size:-pad_size]

plt.figure(figsize=(5,5))
plt.imshow(wall_final, cmap = 'gray')

In [ ]:
## LEGACY: Watershed applied 
# img_bgd_sub = img_smooth.astype(float) - background.astype(float)
# cell_labels, _ = ndi.label(~wall_final_skeleton)

# from skimage.measure import regionprops_table
# props = regionprops_table(cell_labels, properties=['centroid'])

# seeds_mask = np.zeros_like(img_bgd_sub, dtype = bool)

# # For loop through all entries in seeds
# for seed_id in range(np.shape(props['centroid-0'])[0]):
#     seeds_mask[round(props['centroid-0'][seed_id]),round(props['centroid-1'][seed_id])] = 1

# seeds_dil = ndi.binary_dilation(seeds_mask, structure=disk(2))
# plt.imshow(seeds_dil, interpolation='none', cmap='inferno')

# seeds_labeled = ndi.label(seeds_dil)[0]

# from skimage.segmentation import watershed

# ws = watershed(img_bgd_sub, seeds_labeled)

# plt.imshow(img_ch1_crop, interpolation='none', cmap='gray')
# plt.imshow(ws, interpolation='none', cmap='prism', alpha = 0.3)

In [ ]:
# Skeletonization to extract cell wall location
from skimage.morphology import skeletonize

wall_final_skeleton = skeletonize(wall_final)

## Manual Annotation/ Correction with Napari

In [ ]:
# Import napari
import napari

# Start napari viewer
viewer = napari.Viewer()

# Add ch0 raw image
viewer.add_image(img_ch0_crop, colormap = 'gray', name= 'Raw Image Ch0')

# Add ch1 raw image
viewer.add_image(img_ch1_crop, colormap = 'gray', name= 'Raw Image Ch1')

# Add label
viewer.add_labels(wall_final_skeleton)

# Here perform manual annotation in Napari

In [ ]:
# Extract annotation results
last_layer = viewer.layers[2]

# Convert layer to bool array
label_image = last_layer.data.astype(np.bool_)

## Connected Component Labelling

In [ ]:
# (1) Label connected components

# Use the function 'ndi.label' from the 'ndimage' module. 
cell_labels, _ = ndi.label(~label_image)

In [ ]:
# (2) Visulize the result

viewer.add_labels(cell_labels)

## Save results for later processing

In [ ]:
# Import imsave from skimage.io
from skimage.io import imsave

# Save image
imsave(r"example_data/cell_labels.tif", cell_labels.astype(np.uint16))

## Clean Edges

In [ ]:
# (i) Create an image border mask

# We need some way to check if a cell is at the border. For this, we generate a 'mask' of the image border,
# i.e. a Boolean array of the same size as the image where only the border pixels are set to `1` and all 
# others to `0`, like this:
#   1 1 1 1 1
#   1 0 0 0 1
#   1 0 0 0 1
#   1 0 0 0 1
#   1 1 1 1 1
# There are multiple ways of generating this mask, for example by erosion or by array indexing.
# It is up to you to find a way to do it. (Hint: one of the the easiest ways to do this is via scipy.ndimage.binary_dilation.
# check the parameter "border_value")

border_mask = np.zeros(cell_labels.shape, dtype=bool)
border_mask = ndi.binary_dilation(border_mask, border_value=1)

In [ ]:
# (ii) 'Delete' the cells at the border

# 1) Find the cell ROIs that are crossing the border of the image

# Find the border ROI IDs, by first multiply the border_mask by the segmentation mask
border_mask_rois = border_mask * cell_labels

# Then get an array of ROI IDs by finding the unique elements in the array
border_roi_ids = np.unique(border_mask_rois)
border_roi_ids

In [ ]:
# 2) 'Delete' ROIs by their IDs

# Create a copy of the segmentation with np.copy()
clean_cell_labels = np.copy(cell_labels)

# Iterate over ROI IDs on the border and set the those ROIs to background (0)
for roi_id in border_roi_ids:
    
    # Create a mask that contains only the 'current' ROI of the iteration
    roi_mask = cell_labels == roi_id
    
    # Set the position of that roi_mask to background (zero) in the clean_seg
    clean_cell_labels[roi_mask] = 0

In [ ]:
plt.imshow(clean_cell_labels, cmap = 'gray')

## Extracting Quantitative Measures

In [ ]:
# Use the function 'regionprops_table' from the skimage.measure module
from skimage.measure import regionprops_table

# Obtain measurement and save in a parameter
props = regionprops_table(clean_cell_labels, img_ch0_crop, properties=[
                                                                'label',
                                                                'area', 
                                                              'intensity_mean',
                                                              'eccentricity',
                                                              'feret_diameter_max',
                                                              'perimeter',
                                                              'solidity',
                                                             ])

In [ ]:
# Convert props to a pandas dataframe
import pandas as pd

props_df = pd.DataFrame(props)

props_df.head()

In [ ]:
# Save data as csv
props_df.to_csv("leaf_measurement.csv")

## Generate plots about the data

In [ ]:
# Create a histogram of the cell area
plt.figure(figsize=(4,3))
plt.hist(props['area'], bins = 50)
plt.xlabel('Cell Area [pixel]')
plt.ylabel('Count')

In [ ]:
# Create a scatter plot of cell solidity over perimeter
plt.figure(figsize=(5,5))
plt.scatter(props['solidity'],props['perimeter'], edgecolor='k', s=30, alpha=0.5)
plt.xlabel('solidity')
plt.ylabel('Perimeter')

In [ ]:
num_cells = props['label'].shape[0]

In [ ]:
# Create a heat map of cell solidity
heat_map = np.zeros_like(clean_cell_labels, dtype = np.uint8)

min_measure = min(props['solidity'])

max_measure = max(props['solidity'])

for cell_id in range(num_cells):

    cell_mask = clean_cell_labels == props['label'][cell_id]

    measure_8bit = (props['solidity'][cell_id]-min_measure)*255/(max_measure - min_measure)

    heat_map[cell_mask] = measure_8bit
    

In [ ]:
plt.figure(figsize = (5,5))

# Show the heat map. Use a suitable colormap
plt.imshow(heat_map, cmap = 'PRGn')

# Save image as png
plt.savefig('Heatmap.png')